<a href="https://colab.research.google.com/github/ramprakashyadav/OCR/blob/main/redbull_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfplumber

     |████████████████████████████████| 46 kB 2.4 MB/s 
     |████████████████████████████████| 5.6 MB 15.2 MB/s 
     |████████████████████████████████| 4.3 MB 24.7 MB/s 
     |████████████████████████████████| 139 kB 59.1 MB/s 
     |████████████████████████████████| 3.6 MB 60.5 MB/s 
  Created wheel for pdfplumber: filename=pdfplumber-0.6.0-py3-none-any.whl size=33688 sha256=e333d3b600f2daa3162b8a4646be1fefbcd969b6c5ade44cafec1ea50925160f
  Stored in directory: /root/.cache/pip/wheels/58/56/fe/2e93d842ffa9ea97746c1ab253d43502ed61c0689361a0224e
Successfully built pdfplumber
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is in

In [ ]:
import pdfplumber
import os

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/My Drive/redbull_model/

/content/gdrive/My Drive/redbull_model


In [ ]:
INPUT_FOLDER = "/content/gdrive/MyDrive/redbull_model/input_folder/"
OUTPUT_FOLDER = "/content/gdrive/MyDrive/redbull_model/output_text/"

In [ ]:
for fl in os.listdir(INPUT_FOLDER):

  pdf = pdfplumber.open(INPUT_FOLDER+fl)
  pdf_text=pdf.pages[0].extract_text()
  with open(OUTPUT_FOLDER + fl.replace('.pdf','.txt'),'w') as f:
    f.write(pdf_text)

In [ ]:
!pip install -U spacy -q
!python -m spacy info

     |████████████████████████████████| 6.0 MB 8.4 MB/s 
     |████████████████████████████████| 628 kB 64.2 MB/s 
     |████████████████████████████████| 451 kB 43.9 MB/s 
     |████████████████████████████████| 10.1 MB 52.8 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 181 kB 70.4 MB/s 

============================== Info about spaCy ==============================

spaCy version    3.2.1                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.12                        
Pipelines                                      



In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en")
db = DocBin()

In [ ]:
import json
f = open('training_data.json')
TRAIN_DATA = json.load(f)

In [ ]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
          print("Skipping entity")
        else:
          ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy") 

100%|██████████| 885/885 [00:00<00:00, 2209.24it/s]


In [ ]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-02-09 06:30:24,695] [INFO] Set up nlp object from config
[2022-02-09 06:30:24,710] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-02-09 06:30:24,715] [INFO] Created vocabulary
[2022-02-09 06:30:24,717] [INFO] Finished initializing nlp object
[2022-02-09 06:30:25,799] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     63.17    0.00    0.00    0.00    0.00
  2     200        113.33   1037.65   90.38   97.92   83.93    0.90
  6     400         26.38    126.31   91.67   85.94   98.21    0.92
 10     600         13.10    118.60   91.43   97

In [ ]:
nlp_ner = spacy.load("/content/gdrive/MyDrive/redbull_model/model-best")

In [ ]:
doc =nlp_ner('''Bill From
TAX INVOICE
Red Bull India Pvt Ltd.
C/O AWL India Pvt Ltd, Khasra No 393, 394,404-406, Invoice No: 4250010476
Extended Lal Dora Village Invoice Date: 30-11-2021
Alipur, Delhi Ref. No. 194
New Delhi - 110036 Ref Date: 26.11.2021
State Code: 7 GSTIN:07AADCR5842H1ZA Place of Supply: 7
Invoice To
Transporter: 07AAGCA4705P1ZF
Shiv Enterprises
TransporterName: AWL INDIA PVT LTD
6/18, 3rd Floor, Ramesh Nagar
Vehicle No: DL01LM1342
Delhi
LR no: DEL1080005663
Delhi - 110015
Mode of Transport:Road
State Code: 7 GSTIN:07AGPPG6415Q1ZS
Delivery Address
Shiv Enterprises
6/18, 3rd Floor, Ramesh Nagar
Delhi
Delhi - 110015
IRN: c4d2454ace1664443c1d0a29681ae9fc1850776baeeeb70109557a671aa3e159
Rate Per
Serial No Material Description HSN/SAC Qty Gross Amt. Discount Amt. Net Amt. GST Rate % GST Tax Amt. Total Amt. (INR)
Item
IGST: 0.00
Red Bull ED IN Alu Can 24x350ml
CGST: 2,721.00
RB201709 GST: 28
10 22029990 10 BOX 2,063.90 20,639.00 1,200.00 19,439.00 SGST/UGST: 27,214.00
Mfg. Dt.: 14.04.2021-10 CESS: 12
2,721.00
Batch No: 1915069
CESS: 2,333.00
IGST: 0.00
Red Bull IN Sugar Free 250ml WA
CGST: 2,158.00
RB205906 GST: 28
20 22029990 10 BOX 1,661.07 16,610.70 1,200.00 15,410.70 SGST/UGST: 21,575.70
Mfg. Dt.: 15.06.2021-10 CESS: 12
2,158.00
Batch No: 1929483
CESS: 1,849.00
IGST: 0.00
Red Bull Energy Drink IN 250ml
CGST: 19,417.00
RB30925 GST: 28
30 22029990 90 BOX 1,661.07 149,496.30 10,800.00 138,696.30 SGST/UGST: 194,174.30
Mfg. Dt.: 01.05.2021-90 CESS: 12
19,417.00
Batch No: 1918918
CESS: 16,644.00
Total 186,746.00 13,200.00 173,546.00 69,418.00 242,964.00
TCS 0.00
Total Amount 242,964.00
Amount in words:INR Two Lakh Forty Two Thousand Nine Hundred and Sixty Four Only
Payment Terms - Due immediately
Declaration:
No credit of additional duty of custom (ADC) is admissible to the buyer. We declare that this document shows the actual price of the goods described and that all particular are true and
correct. 
We hereby certify that food/foods mentioned in this document is / are warranted to be of the nature and quality which it/these purports/purport to be.
No. of Box: 110
Gross weight - 765.360
FSSAI Lic. No - 13314002000283
For Red Bull India Pvt. Ltd.
Authorised signatory
Red Bull India Pvt. Ltd.
Registered Office Address: 1st Floor, B Wing, 215 Atrium, Kanakia Spaces, Chakala, Andheri Kurla Road, Andheri (E) Mumbai-400093. www.redbull.in
Tel:+91-22-66180888, Fax:+91-22-66180808, Email: info.in@redbull.com, CIN: U51909MH2007FTC169580 PAN: AADCR5842H''')

In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True)

In [ ]:
import pandas as pd

In [ ]:
all_data =[]
for fl in os.listdir(INPUT_FOLDER):
    print(fl)
    pdf_obj = pdfplumber.open(INPUT_FOLDER + fl)
    for page in pdf_obj.pages:
        tables = page.extract_tables()
        page_txt = page.extract_text()
        # print(page_txt)
        for table in tables:
            if 'INVOICE' in ''.join(table[0]):
                invoice_no =  table[1][0].split('\n')[0].replace('Invoice No:','').strip()
                print(invoice_no)
            if 'Serial No' in ''.join(table[0]) or 'Item' in ''.join(table[0]):
                table_df = pd.DataFrame(table[1::],columns=table[0])
                # print(table_df.columns)
                for idx, row in table_df.iterrows():
                    if row['Serial No'].isnumeric():
                        # print(type(row['Serial No']))
#                         print(idx)
                        if idx == 0:
                            doc = nlp_ner(page_txt)
                            all_label_text = []
                            for ent in doc.ents:
                              print(ent.label_)
                              if ent.label_ == "COMPANY NAME":
                                company_name = ent.text
                              if ent.label_ == "INVOICE DATE":
                                invoice_date = ent.text
                              if ent.label_ == "COMPANY GSTIN NUMBER":
                                company_gstin = ent.text
                              if ent.label_ == "VEHICLE NO":
                                vehicle_no = ent.text
                              if ent.label_ == "BUYER GSTIN NUMBER":
                                buyer_gstin = ent.text
                              if ent.label_ == "CIN NO":
                                cin_no = ent.text
                              if ent.label_ == "COMPANY PAN NO":
                                company_pan = ent.text
                            tot_row = table_df[table_df['Serial No'] == 'Total']
                            receipt_tot_amt = table_df[table_df['Serial No'] == 'Total Amount']['Total Amt. (INR)'].values[0]
                            all_data.append([invoice_no,company_name,invoice_date,company_gstin,vehicle_no,buyer_gstin,cin_no,company_pan,receipt_tot_amt, tot_row['Gross Amt.'].values[0], tot_row['Discount Amt.'].values[0], tot_row['Net Amt.'].values[0], tot_row['GST Tax Amt.'].values[0], row['Serial No'], row['Material Description'], row['HSN/SAC'], row['Qty'], row['Rate Per\nItem'], row['Gross Amt.'], row['Discount Amt.'], row['Net Amt.'], row['GST Rate %'], row['GST Tax Amt.']])
                        else:
                            all_data.append(['','','','','','','','','','','','','', row['Serial No'], row['Material Description'], row['HSN/SAC'], row['Qty'], row['Rate Per\nItem'], row['Gross Amt.'], row['Discount Amt.'], row['Net Amt.'], row['GST Rate %'], row['GST Tax Amt.'], row['Total Amt. (INR)']  ])
                    
                print("_________________________")
                                             

GRANARYWHOLESALEPRIVATELIMITE_4203304350.pdf
4203304350
COMPANY NAME
INVOICE DATE
COMPANY GSTIN NUMBER
VEHICLE NO
BUYER GSTIN NUMBER
CIN NO
COMPANY PAN NO
_________________________
RELIANCERETAILLIMITED_4250010632.pdf
4250010632
COMPANY NAME
INVOICE DATE
COMPANY GSTIN NUMBER
VEHICLE NO
BUYER GSTIN NUMBER
CIN NO
COMPANY PAN NO
_________________________
P.N.Arora_Co._4250010669.pdf
4250010669
COMPANY NAME
INVOICE DATE
COMPANY GSTIN NUMBER
VEHICLE NO
BUYER GSTIN NUMBER
CIN NO
COMPANY PAN NO
_________________________
RajsonsDistributors_4250010501.pdf
4250010501
COMPANY NAME
INVOICE DATE
COMPANY GSTIN NUMBER
VEHICLE NO
BUYER GSTIN NUMBER
CIN NO
COMPANY PAN NO
_________________________
RajuEggSales_4250010628.pdf
4250010628
COMPANY NAME
INVOICE DATE
COMPANY GSTIN NUMBER
VEHICLE NO
BUYER GSTIN NUMBER
CIN NO
COMPANY PAN NO
_________________________
CPWHOLESALEINDIAPRIVATELIMITE_4250010482.pdf
4250010482
COMPANY NAME
INVOICE DATE
COMPANY GSTIN NUMBER
VEHICLE NO
BUYER GSTIN NUMBER
CIN NO
COMPAN

In [ ]:
out_df = pd.DataFrame(all_data, columns= ['Invoice No','COMPANY NAME','INVOICE DATE','COMPANY GSTIN NUMBER','VEHICLE NO','BUYER GSTIN NUMBER','CIN NO','COMPANY PAN NO', 'Invoice Total Amt', 'Total Gross AMt.', 'Total Discount Amt.', 'Total Net Amt.', 'Total GST Tac Amt.','Serial No', 'Material Description', 'HSN/SAC', 'Qty', 'Rate Per\nItem',
       'Gross Amt.', 'Discount Amt.', 'Net Amt.', 'GST Rate %', 'GST Tax Amt.',
       'Total Amt. (INR)'])

In [ ]:
out_df

,Invoice No,COMPANY NAME,INVOICE DATE,COMPANY GSTIN NUMBER,VEHICLE NO,BUYER GSTIN NUMBER,CIN NO,COMPANY PAN NO,Invoice Total Amt,Total Gross AMt.,Total Discount Amt.,Total Net Amt.,Total GST Tac Amt.,Serial No,Material Description,HSN/SAC,Qty,Rate Per\nItem,Gross Amt.,Discount Amt.,Net Amt.,GST Rate %,GST Tax Amt.,Total Amt. (INR)
0,4203304350,RedBull India Pvt. Ltd,06-09-2021,GSTIN:09AADCR5842H1Z6,UP32T4543,GSTIN:09AAHCG7552R1ZP,U51909MH2007FTC169580,AADCR5842H,"1,032,752.00","824,680.00","87,000.00","737,680.00","295,072.00",10,Red Bull ED IN 250ml 4Pack PMC\nRB202075\nMfg....,22029990,250 BOX,"1,603.29","400,822.50","33,000.00","367,822.50",GST: 28\nCESS: 12,"IGST: 0.00\nCGST: 51,495.00\nSGST/UGST:\n51,49...",None
1,,,,,,,,,,,,,,20,Red Bull Energy Drink IN 250ml\nRB30925\nMfg. ...,22029990,250 BOX,"1,695.43","423,857.50","54,000.00","369,857.50",GST: 28\nCESS: 12,"IGST: 0.00\nCGST: 51,780.00\nSGST/UGST:\n51,78...","517,800.50"
2,4250010632,Red Bull India Pvt,16-12-2021,GSTIN:07AADCR5842H1ZA,DL01LX2509,GSTIN:08AABCR1718E1ZP,U51909MH2007FTC169580,AADCR5842H,"204,480.00","161,657.00","15,600.00","146,057.00","58,422.96",10,Red Bull Energy Drink IN 250ml\nRB30925\nMfg. ...,22029990,100 BOX,"1,616.57","161,657.00","15,600.00","146,057.00",GST: 28\nCESS: 12,"IGST: 40,895.96\nCGST: 0.00\nSGST/UGST: 0.00\n...",None
3,4250010669,Red Bull India Pvt,21-12-2021,GSTIN:07AADCR5842H1ZA,DL01LAD3341,GSTIN:07AEJPA7529M1Z7,U51909MH2007FTC169580,AADCR5842H,"2,407,031.60","1,719,308.60",0.00,"1,719,308.60","687,723.00",10,Red Bull ED IN Alu Can 24x350ml\nRB201709\nMfg...,22029990,865 BOX,"1,987.64","1,719,308.60",0.00,"1,719,308.60",GST: 28\nCESS: 12,"IGST: 0.00\nCGST: 240,703.00\nSGST/UGST:\n240,...",None
4,4250010501,Red Bull India Pvt,03-12-2021,GSTIN:07AADCR5842H1ZA,DL01LM1342,GSTIN:07AAAFR5665A1ZG,U51909MH2007FTC169580,AADCR5842H,"419,796.60","323,854.60","24,000.00","299,854.60","119,942.00",10,Red Bull ED IN Alu Can 24x350ml\nRB201709\nMfg...,22029990,20 BOX,"2,063.90","41,278.00","2,400.00","38,878.00",GST: 28\nCESS: 12,"IGST: 0.00\nCGST: 5,443.00\nSGST/UGST:\n5,443....",None
5,,,,,,,,,,,,,,20,Red Bull ED IN 250ml 4Pack PMC\nRB202075\nMfg....,22029990,180 BOX,"1,569.87","282,576.60","21,600.00","260,976.60",GST: 28\nCESS: 12,"IGST: 0.00\nCGST: 36,537.00\nSGST/UGST:\n36,53...","365,367.60"
6,4250010628,Red Bull India Pvt,16-12-2021,GSTIN:07AADCR5842H1ZA,DL01LM1342,GSTIN:07AAOPA8256A1ZX,U51909MH2007FTC169580,AADCR5842H,"507,466.22","362,113.26",0.00,"362,113.26","144,846.00",10,Red Bull ED IN Alu Can 24x250ml\nWA\nRB226600\...,22029990,218 BOX,"1,661.07","362,113.26",0.00,"362,113.26",GST: 28\nCESS: 12,"IGST: 0.00\nCGST: 50,696.00\nSGST/UGST:\n50,69...",None
7,4250010482,Red Bull India Pvt,01-12-2021,GSTIN:07AADCR5842H1ZA,HR39D9497,GSTIN:06AAGCC7979R1ZL,U51909MH2007FTC169580,AADCR5842H,"1,040,018.00","850,870.00","108,000.00","742,870.00","297,147.60",10,Red Bull Energy Drink IN 250ml\nRB30925\nMfg. ...,22029990,500 BOX,"1,701.74","850,870.00","108,000.00","742,870.00",GST: 28\nCESS: 12,"IGST: 208,003.60\nCGST: 0.00\nSGST/UGST: 0.00\...",None
8,4250010498,Red Bull India Pvt,02-12-2021,GSTIN:07AADCR5842H1ZA,DL01LAD0134,GSTIN:06AAOFM8498A1ZW,U51909MH2007FTC169580,AADCR5842H,"946,283.83","729,241.50","54,000.00","675,241.50","270,096.62",10,Red Bull ED IN 250ml 4Pack PMC\nRB202075\nMfg....,22029990,200 BOX,"1,569.87","313,974.00","24,000.00","289,974.00",GST: 28\nCESS: 12,"IGST: 81,192.72\nCGST: 0.00\nSGST/UGST: 0.00\n...",None
9,,,,,,,,,,,,,,20,Red Bull Energy Drink IN 250ml\nRB30925\nMfg. ...,22029990,250 BOX,"1,661.07","415,267.50","30,000.00","385,267.50",GST: 28\nCESS: 12,"IGST: 107,874.90\nCGST: 0.00\nSGST/UGST: 0.00\...","539,374.50"


In [ ]:
out_df.to_csv(OUTPUT_FOLDER+'test_redbull232.csv')